In [1]:
# Nonlinear MPC test using kinematic bicycle model for car
# We want to use the kinematic bicycle model from this paper (Figure 3)
# https://link-springer-com.stanford.idm.oclc.org/article/10.1007/s13177-020-00226-1#Tab4

# which has states:
# x, y = positional coordinates in inertial frame
# ψ = angle of car centerline in inertial frame
# ̇ψ = change of angle
# β = angle between car centerline and velocity vector
# ̇β = change of angle

# and has the equations:
# ̇x = v cos(ψ + β)
# ̇y = v sin(ψ + β)
# ̇v = a
# ̇ψ = v / lᵣ sin(β)
# β = arctan(lᵣ/(lf + lᵣ) tan(δf))
# ̇β = v/(lf + lᵣ) tan(δf) - v/lᵣ sin(β)
# ̇δf = ω

# and we want to solve it. Define the model:


The model has state and input:
\begin{gather*}
z = \begin{bmatrix}x\\ y\\ v\\ \psi\\ \beta\end{bmatrix},\quad
u = \begin{bmatrix}a\\ \delta_f\end{bmatrix},\quad
\end{gather*}
where the two control signals are $a$, the acceleration command, and $\delta_f$, the front steering angle (in radians).
The nonlinear model is:
\begin{gather*}
\begin{bmatrix}
\dot x\\ \dot y\\ \dot v\\ \dot\psi\\ \dot\beta
\end{bmatrix} =
\begin{bmatrix}
v\cos(\psi + \beta)
\\
v\sin(\psi + \beta)
\\
a
\\
\frac{v}{l_r}\sin(\beta)
\\
\frac{v}{l_f + l_r}\tan(\delta_f) - \frac{v}{l_r}\sin(\beta)
\end{bmatrix}
\end{gather*}

## Casadi setup
From http://casadi.sourceforge.net/v3.4.4/users_guide/casadi-users_guide.pdf

In [2]:
import numpy as np
import casadi
from direct_collocation import MpcProblem
from KinematicBicycleCar import KinematicBicycleCar

In [3]:
# Set up a visualization

import matplotlib.pyplot as plt
import matplotlib as mpl



In [4]:
# Acceleration bounds from 
# https://link-springer-com.stanford.idm.oclc.org/article/10.1007/s13177-020-00226-1#Sec3

# TODO: Fix lower/upper bounds to sample from functions for x, y.
def lowerbounds(varname, i, k):
    if varname == 'z':
        limits = [-1.0,
                  -1.0,
                  0.0,
                  -np.pi/4,
                  -np.pi/4]
        return limits[i]
    elif varname == 'u':
        limits = [-5, -np.pi/4] # >= -1.70 preferred
        return limits[i]
    else:
        print("HUGE ERROR: lowerbounds doesn't understand variable named "+varname)
        return -np.inf
    
def upperbounds(varname, i, k):
    if varname == 'z':
        limits = [20.0,
                  1.0,
                  20.0,
                  np.pi/4,
                  np.pi/4]
        return limits[i]
    elif varname == 'u':
        limits = [2.5, np.pi/4] # <= 0.84 preferred
        return limits[i]
    else:
        print("HUGE ERROR: upperbounds doesn't understand variable named "+varname)
        return np.inf


In [5]:
car = KinematicBicycleCar(N=30, step=0.05)
        
ic = [0.0, 0.0, 5.5, np.pi/8, 0.0]
car.set_initial(ic)

# Cost
cost = (car.dae.x[0][2]-5)**2

In [ ]:
    
mpcprob = MpcProblem(car,     # casadi.DaeBuilder()
                     cost,        # casadi symbolic objective
                     lowerbounds, # given name of state/control variable, return lower bound
                     upperbounds, # given name of state/control variable, return upper bound
                     )
uk = [0.0,0.0]

n_runs = 20
x_plan = np.empty((mpcprob.model.n,n_runs+1))
u_plan = np.empty((mpcprob.model.m,n_runs))
x_true = np.empty((mpcprob.model.n,n_runs+1))
x_true[:,0] = ic
x_plan[:,0] = ic

for k in range(n_runs):
    xk_opt, uk_opt = mpcprob.run(x_true[:,k])
    for i in range(5):
        x_plan[i,k+1] = xk_opt[i][1]
    for i in range(2):
        u_plan[i,k] = uk_opt[i][1]
    x_true[:,k+1] = mpcprob.simulate(x_true[:,k], u_plan[:,k])
    
    # hack, test changing cost
    if k > 10:
        mpcprob.set_cost(100*car.dae.x[0][2]**2)

#for k in range(1,n_runs):
    #print("True:", x_true[:,k])
    #print("Pred:", x_plan[:,k])
    #print("Error:", np.linalg.norm(x_true[:,k]-x_plan[:,k],2))


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     3520
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      660

Total number of variables............................:      660
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      660
                     variables with only upper bounds:        0
Total number of equa

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     3520
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      660

Total number of variables............................:      660
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      660
                     variables with only upper bounds:        0
Total number of equality constraints.................:      600
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

  16  1.8500836e-03 1.36e-05 2.19e-03  -3.8 1.68e+00    -  9.13e-01 1.00e+00h  1
  17  1.3637742e-03 9.53e-06 5.61e-05  -3.8 5.61e-01  -4.0 1.00e+00 1.00e+00h  1
  18  1.1304881e-03 3.54e-07 1.21e-04  -5.7 6.55e-01  -4.5 9.18e-01 1.00e+00h  1
  19  1.0489353e-03 3.57e-07 4.71e-09  -5.7 4.56e-01    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  20  1.0418144e-03 6.77e-10 5.16e-07  -8.6 4.75e-02  -5.0 9.92e-01 1.00e+00h  1
  21  1.0416578e-03 1.05e-09 3.05e-08  -8.6 8.24e-03  -5.4 1.00e+00 1.00e+00h  1
  22  1.0416585e-03 1.59e-14 3.39e-08  -8.6 6.71e-06  -2.3 1.00e+00 1.00e+00h  1
  23  1.0416584e-03 2.25e-05 1.15e-11  -8.6 3.98e-02    -  1.00e+00 1.00e+00h  1
  24  1.0416584e-03 3.13e-10 4.95e-08  -8.6 8.81e-05  -3.3 1.00e+00 1.00e+00h  1
  25  1.0416584e-03 1.90e-14 1.34e-08  -8.6 1.12e-06  -1.9 1.00e+00 1.00e+00h  1
  26  1.0416568e-03 3.94e-09 2.74e-10  -9.0 4.97e-04    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 2

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     3520
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      660

Total number of variables............................:      660
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      660
                     variables with only upper bounds:        0
Total number of equality constraints.................:      600
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     3520
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      660

Total number of variables............................:      660
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      660
                     variables with only upper bounds:        0
Total number of equality constraints.................:      600
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     3520
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      660

Total number of variables............................:      660
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      660
                     variables with only upper bounds:        0
Total number of equality constraints.................:      600
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

  47  9.6468711e+02 1.59e-14 1.10e-05  -8.6 3.60e-04  -1.5 1.00e+00 1.00e+00h  1
  48  9.6468711e+02 1.44e-14 8.37e-06  -8.6 3.07e-04  -1.6 1.00e+00 1.00e+00h  1
  49  9.6468711e+02 2.30e-14 8.15e-06  -8.6 4.20e-05  -0.7 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  50  9.6468711e+02 1.18e-14 7.67e-06  -8.6 4.45e-05  -0.8 1.00e+00 1.00e+00h  1
  51  9.6468711e+02 1.30e-14 7.21e-06  -8.6 4.71e-05  -0.8 1.00e+00 1.00e+00h  1
  52  9.6468711e+02 1.56e-14 7.20e-06  -8.6 2.20e-06   0.5 1.00e+00 1.00e+00h  1
  53  9.6468711e+02 1.44e-14 7.02e-06  -8.6 1.93e-05  -0.4 1.00e+00 1.00e+00h  1
  54  9.6468711e+02 1.28e-14 6.82e-06  -8.6 2.11e-05  -0.5 1.00e+00 1.00e+00h  1
  55  9.6468711e+02 1.85e-14 5.41e-06  -8.6 1.51e-04  -1.4 1.00e+00 1.00e+00h  1
  56  9.6468711e+02 1.30e-14 5.53e-06  -8.6 9.02e-07   0.8 1.00e+00 1.00e+00h  1
  57  9.6468711e+02 1.54e-14 5.52e-06  -8.6 3.38e-07   1.2 1.00e+00 1.00e+00h  1
  58  9.6468711e+02 1.35e-14

In [ ]:
u_plt, u_ax1, u_ax2 = car.plot_u(u_plan, mpcprob.u_opt)
# control limits
x_plt, x_ax = car.plot_x(x_plan, mpcprob.x_opt)
x_ax.plot([0.0,10.0], [-1,-1]) # Road boundary (lower)
x_ax.plot([0.0,10.0], [1,1]) # (upper)
plt.show()

In [ ]:
print("Final state:")
print("(x,y):", x_plan[0,-1], x_plan[1,-1])
print("v    :", x_plan[2])

In [ ]:
# This is the road to my parents' house, distances in meters.

road_left = np.array([
[2.519, 117.514],
[10.68, 117.192],
[22.303, 116.549],
[30.712, 115.585],
[40.357, 112.691],
[50.744, 107.226],
[50.249, 98.224],
[48.765, 84.721],
[47.529, 74.754],
[47.158, 64.466],
[47.034, 53.535],
[47.529, 41.318],
[48.024, 31.994],
[48.518, 22.028],
[58.41, 22.671],
[68.303, 23.635],
[77.453, 23.153],
])

road_right = np.array([
[2.519, 109.155],
[10.68, 109.316],
[22.427, 108.19],
[30.094, 107.708],
[38.626, 105.297],
[42.83, 102.403],
[43.325, 98.545],
[42.583, 86.007],
[41.222, 75.719],
[40.852, 65.431],
[40.357, 53.856],
[40.852, 41.318],
[41.346, 30.869],
[45.179, 14.955],
[57.916, 14.312],
[68.056, 15.276],
[77.206, 14.312],
])

plt.scatter(road_left[:,0], road_left[:,1])
plt.scatter(road_right[:,0], road_right[:,1])

In [ ]:
INITIAL_CONDITIONS = np.array([
    (road_right[0,0] + road_left[0,0])/2.0, # x, car starts in middle of road
    (road_right[0,1] + road_left[0,1])/2.0, #y, same
    11.0, # v, about 25 MPH = 11.17 m/s. This is a bit too fast.
    -np.pi/8, # psi, car is a bit crooked
    0.0 # beta
])

N_ROAD_POINTS = 17


In [ ]:
fig1, ax = plt.subplots(1, 1, figsize=(4,8))
ax.scatter(road_left[:,0], road_left[:,1])
ax.scatter(road_right[:,0], road_right[:,1])

left  = np.polynomial.polynomial.Polynomial.fit(road_left[-4:,0], road_left[-4:,1],3)
right = np.polynomial.polynomial.Polynomial.fit(road_right[-4:,0], road_right[-4:,1],3)
limit_start = (road_left[-4] + road_right[-4])/2.0
limit_end = (road_left[-1] + road_right[-1])/2.0


x = np.linspace(limit_start[0], limit_end[0], 10)
ax.plot(x, left(x))
ax.plot(x, right(x))

In [ ]:
#
#
#
#
#
#
#
#
#
#

In [ ]:
# Acceleration bounds from 
# https://link-springer-com.stanford.idm.oclc.org/article/10.1007/s13177-020-00226-1#Sec3

# TODO: Fix lower/upper bounds to sample from functions for x, y.
def lowerbounds(varname, i, k):
    if varname == 'z':
        limits = [limit_start[0]+-5*0.05*10,
                  min(left(k*5*0.05+limit_start[0]), right(k*5*0.05+limit_start[0]))*10,
                  0.0,
                  -np.pi/4,
                  -np.pi/4]
        return limits[i]
    elif varname == 'u':
        limits = [-1.70, -np.pi/4]
        return limits[i]
    else:
        print("HUGE ERROR: lowerbounds doesn't understand variable named "+varname)
        return -np.inf
    
def upperbounds(varname, i, k):
    if varname == 'z':
        limits = [limit_start[0]+5*0.05*10,
                  max(left(k*5*0.05+limit_start[0]), right(k*5*0.05+limit_start[0]))*10,
                  20.0,
                  np.pi/4,
                  np.pi/4]
        return limits[i]
    elif varname == 'u':
        limits = [0.84, np.pi/4]
        return limits[i]
    else:
        print("HUGE ERROR: upperbounds doesn't understand variable named "+varname)
        return np.inf

    
upper_bound = np.empty((250,2))
lower_bound = np.empty((250,2))
for k in range(250):
    upper_bound[k,0] = upperbounds('z',0,k)
    upper_bound[k,1] = upperbounds('z',1,k)
    
    lower_bound[k,0] = lowerbounds('z',0,k)
    lower_bound[k,1] = lowerbounds('z',1,k)
    
plt.plot(upper_bound[:,0], upper_bound[:,1], label="Upper")
plt.plot(lower_bound[:,0], lower_bound[:,1], label="Lower")
plt.legend()

In [ ]:
#
#
#
#
#
#
#
#
#
#

In [ ]:
car = KinematicBicycleCar(N=30, step=0.05)
        
ic = [0.0, 0.0, 5.5, np.pi/8, 0.0]
car.set_initial(ic)

# Cost
cost = (car.dae.x[0][2]-5)**2

In [ ]:
    
mpcprob = MpcProblem(car.dae,         # casadi.DaeBuilder()
                     cost,        # casadi symbolic objective
                     lowerbounds, # given name of state/control variable, return lower bound
                     upperbounds, # given name of state/control variable, return upper bound
                     N    = 30,   # MPC horizon (steps)
                     step = 0.05, # Time step (seconds)
                     )
uk = [0.0,0.0]

n_runs = 20
x_plan = np.empty((mpcprob.n,n_runs+1))
u_plan = np.empty((mpcprob.m,n_runs))
x_true = np.empty((mpcprob.n,n_runs+1))
x_true[:,0] = ic
x_plan[:,0] = ic

for k in range(n_runs):
    xk_opt, uk_opt = mpcprob.run(x_true[:,k])
    for i in range(5):
        x_plan[i,k+1] = xk_opt[i][1]
    for i in range(2):
        u_plan[i,k] = uk_opt[i][1]
    x_true[:,k+1] = mpcprob.simulate(x_true[:,k], u_plan[:,k])
    
    # hack, test changing cost
    if k > 30:
        mpcprob.set_cost(car.dae.x[0][2]**2)
    # Bring to a stop (hack)
    if k >= n_runs-1:
        mpcprob.set_final_constraint(0.0,0.0)

#for k in range(1,n_runs):
    #print("True:", x_true[:,k])
    #print("Pred:", x_plan[:,k])
    #print("Error:", np.linalg.norm(x_true[:,k]-x_plan[:,k],2))

In [ ]:
u_plt, u_ax1, u_ax2 = car.plot_u(u_plan, mpcprob.u_opt)
# control limits
x_plt, x_ax = car.plot_x(x_plan, mpcprob.x_opt)
x_ax.plot([0.0,10.0], [-1,-1]) # Road boundary (lower)
x_ax.plot([0.0,10.0], [1,1]) # (upper)
plt.show()